In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

In [ ]:
csv_files = ["bomojobrandindices.csv", "boxofficemojotopgenres.tsv", 
             "boxofficemojotopfranchises.tsv", "boxofficemojobrandindices.tsv", 
             "boxofficemojointernationaltop1000.tsv"]

data = {}
# прочтение файлов, т.к. они все имеют разделение табуляцией, в качестве сепаратора указываю табуляцию.  Содержание двух файлов  
# bomojobrandindices.csv   boxofficemojotopgenres.tsv имеют одинаковое содержание 
for file in csv_files:
 
    df = pd.read_csv(file, sep="\t")
    data[file.split('.')[0]] = df.copy()
    print(f"Содержимое файла {file}:")
    print(df.head(10))
    
    print("\n")
#print(data)

bomojobrandindices.csv содержит следующие признаки(колонки) названия брендов/ международная выручка/ сколько релизов всего/ самый успешный релиз/ выручка по самому успешному релизу

boxofficemojotopgenres.tsv:  жанр/ международная выручка/ сколько фильмов/ самый успешный фильм/ выручка по самому успешному фильму

boxofficemojotopfranchises.tsv: название франшизы/ международная выручка/ сколько релизов всего/ самый успешный релиз/ выручка по самому успешному релизу

boxofficemojobrandindices.tsv: бренд(правообладатель) /международная выручка/ сколько релизов всего/ самый успешный релиз/ выручка по самому успешному релизу

boxofficemojointernationaltop1000.tsv: топовый фильм/ международная(общая)выручка / локальная выручка/ %локальной выручки от общей/ зарубежная выручка / % зарубежной выручки от общей 


Цель: определить наиболее успешную франшизу по выручке на один фильм и сделать вывод какой тип франшизы наиболее привлекателен
потенциально (комиксы, мультфильмы и т.п.).

In [ ]:
for file_name, df in data.items():
    print(f"Для файла {file_name}:")
    # проверка типов данных и приведение их к нужному типу
    print("Типы данных до обработки:")
    print(df.dtypes)
    
    # проверка наличия пропущенных значений
    print("\nПропущенные значения:")
    print(df.isnull().sum())

    # проверка данных на наличие дубликатов
    print("\nдубликаты:")
    print(df.duplicated().sum())

    print("\n")

В анализируемых данных нет пропусков и дубликатов.

In [ ]:
for file_name, df in data.items():
    # рассчет основных статистических показателей
    print(f"Основные статистические показатели для файла {file_name}:")
    print(df.describe())
    print("\n")

In [ ]:
for file_name, df in data.items():
    print(f"Для файла {file_name}:")

    # гистограммы числовых переменных
    numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns
    df[numeric_cols].hist(figsize=(10, 6), bins=20)
    plt.suptitle("Гистограммы числовых переменных")
    plt.show()

    # диаграммы рассеяния для пар числовых переменных
    sns.pairplot(df[numeric_cols])
    plt.title("Диаграммы рассеяния для пар числовых переменных")
    plt.show()

    # столбчатые диаграммы для категориальных переменных
    categorical_cols = df.select_dtypes(include=['object']).columns
    for col in categorical_cols:
        sns.countplot(data=df, x=col)
        plt.title(f"Столбчатая диаграмма для переменной '{col}'")
        plt.xticks(rotation=45)
        plt.show()

    print("\n")

по диаграммам сложно что-то проанализировать, столбчатые диаграммы для категориальных не дают какой либо стоящей информации, для числовых переменных более наглядны.  
boxofficemojointernationaltop1000.tsv (с колонками - Rank: Порядковый номер фильма в рейтинге; Title: Название фильма; Worldwide Lifetime Gross: Общий мировой сбор фильма за всё время проката в долларах США; Domestic Lifetime Gross: Сборы фильма на внутреннем (внутри страны) рынке проката в долларах США; Domestic %: Процент сборов фильма на внутреннем рынке проката от общих сборов; Foreign Lifetime Gross: Сборы фильма на международном рынке проката в долларах США; Foreign %: Процент сборов фильма на международном рынке проката от общих сборов; Year: Год выхода фильма. ) кажется наиболее наглядно отражается на гистограммах рассеяния и подходит для построения карты корреляции.    

In [ ]:
df = data['boxofficemojointernationaltop1000'].copy()

# используем только числовые признаки для построения карты корреляции
numeric_cols = df.select_dtypes(include=['int64', 'float64'])

# строим матрицу корреляции
corr_matrix = numeric_cols.corr()

# создаем тепловую карту корреляции
plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=.5)
plt.title("Карта корреляции числовых признаков")
plt.show()

между Worldwide Lifetime Gross и Domestic Lifetime Gross имеют достаточно высокий коэфициент корреляции, что ожидаемо, переменные зависимы и  общий мировой сбор фильма и сборы фильма внутри страны могут быть связаны. Так же эти переменные коррелируют с рангом фильма в рейтинге, с возрастанием ранга популярность фильма парает соответственно будет отрицательная корреляция с кассовыми сборами.

#### Определяем наиболее успешную франшизу по выручке на один фильм

In [ ]:
# копируем датафрейм
df = data['boxofficemojotopfranchises'].copy()

# создаем новый столбец с отношением Total к Releases
df['Total_Per_Release'] = df['Total'] / df['Releases']

# находим индекс строки с максимальным значением в столбце 'Total_Per_Release'
max_total_per_release_index = df['Total_Per_Release'].idxmax()

# получаем значение в столбце "Franchise" для строки с максимальным значением в столбце 'Total_Per_Release'
franchise_with_max_total_per_release = df.loc[max_total_per_release_index, 'Franchise']

print("Франшиза с наибольшим значением Total/Releases:",
      franchise_with_max_total_per_release)

#### Определяем какой тип жанр потенциально наиболее привлекателен

In [ ]:
# копируем датафрейм
df = data['boxofficemojotopgenres'].copy()

# вычисляем средний доход на фильм для каждого жанра
#genre_stats['Average_Total'] = genre_stats['Total'] / genre_stats['Titles']
df['Average_Total'] = df['Total'] / df['Titles']

# сортируем значения среднего дохода на фильм по убыванию
genre_stats_sorted = df.sort_values(
    by='Average_Total', ascending=False)

print(genre_stats_sorted[['Genre', 'Average_Total']].head(10))

По распределению доходности жанров можно предположить, что для успешности фильма сюжет должен иметь основную линию связанную с космосом, главный герой из разряда "супергероев" с описанием его биографии и лучше добавить мифических существ/монстров/динозавров/драконов.